# 제로베이스데이터사이언스 스쿨 원데이 프로젝트
> ## 세계 테러 데이터 분석 - SQL 사용하기

---

#### 총 8문제 100점 만점 

- 문제 1 10점  
- 문제 2,3,5,6,7,8번 각 10점 
- 문제 4번 30점
- 총점 100점 

#### 유의사항

- 모든 정답 코드는 jupyter notebook 파일에 작성해주세요. 
- 1번 문제는 정답 코드를 그대로 공개해드립니다. 따라서 진행해주세요. 
- 그 외 문제들은 참고하실 수 있도록 주석과 출력 코드를 보여드립니다. 
- 제출시, 반드시 결과가 실행되어 있어야 합니다. 미실행 또는 오류시 감점입니다. 
- 제출시, [DS]sql3_choyongha.ipynb 형식에 맞춰 제출해주세요.


---

여러분은 이전 과제에서 세계 테러 데이터 분석 프로젝트를 진행하였습니다. 

이번 시간에는 분석할 데이터를 Database 로 변환하고, 분석에 필요한 데이터를 저장된 Database 에서 쿼리하여 가져와 보도록 하겠습니다. 

#### 작업환경

- aws rds 사용시: 이전 과제에서 생성한 AWS RDS (MySQL) 의 Database 및 사용자를 사용하세요. 
  - Database Name: oneday 
  - User Name / Password: oneday / 1234
- local 사용시: 로컬 환경에서 생성한 Database 및 사용자를 사용하세요.
  - 위와 동일하게 생성해주세요

## 원본 Data Download
- https://www.kaggle.com/datasets/START-UMD/gtd

---

문제 1.  csv 파일에 저장된 세계 테러 데이터를 하나의 테이블에 저장하세요. 

- globalterrorismmdb_0718.csv - https://www.kaggle.com/datasets/START-UMD/gtd
- pandas의 to_sql 함수 사용 (힌트 : create_engine)
- Table name : origin_terror_data 

제출 1. Table 생성 및 조회 결과 : DESC origin_terror_data, SELECT  COUNT(*) FROM origin_terror_data

제출 2. 관련 코드 및 실행 결과 (ipynb)



In [2]:
# pip install sqlalchemy

In [8]:
# pip install pymysql

In [1]:
import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import configparser

# 데
import warnings
warnings.filterwarnings('ignore')

In [3]:
##############################################################################################
#  문제 1. 원본 데이터 (csv)를 데이터베이스로 변환하고 확인하기 
#  globalterrorismdb_0718dist.csv -> origin_terror_data (table)
#  Pandas 의 to_sql 사용 
##############################################################################################

##############################################################################################
#  pip install pymysql
#  pip install SQLAlchemy Flask-SQLAlchemy
##############################################################################################

import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import configparser

# 데이터 불러오기
df = pd.read_csv('../sql3/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

# DB 접속 엔진 객체 생성
user = 'root'
password = 'mint'
host = 'localhost'
port = 3306
database = 'oneday'

# DB 연결 객체 생성
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4')

# DB 테이블 명(생성될 테이블 이름)
table_name = "origin_terror_data"

# DB에 DataFrame 적재
df.to_sql(index = False, 
          name = table_name,
          con = engine,
          if_exists = 'append',
          method = 'multi', 
          chunksize = 10000)

print(1)

C:\Users\HoJin\AppData\Local\Temp\ipykernel_9676\2343190209.py:19: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../sql3/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')


1


In [2]:
import mysql.connector

In [13]:
remote = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = 'mint',
    database = 'oneday'
)
cur = remote.cursor(buffered=True)
cur.execute('SELECT COUNT(*) FROM origin_terror_data')
records = cur.fetchone()  # fetchone을 사용하여 단일 결과를 가져옴
print(records)

(181691,)


문제 2. origin_terror_data 에서 region, country 관련 데이터는 code 와 txt (name) 속성으로 정의되어 있습니다. 

문제 2-1. Region 및 Country 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 
- Region 과 Country 데이터 사이의 관계를 파악하여 Foreign Key 를 설정하세요.

참고> 
- Region.region_code     = origin_terror_data.region
- Region.region_name     = origin_terror_data.region_txt
- Country.country_code   = origin_terror_data.country
- Country.country_name   = origin_terror_data.country_txt


In [3]:
##############################################################################################
#  문제 2. region / country / city 데이터 추출하여 데이터베이스로 변환하기 
#  중복을 제거한 code - name 값 
#  region - country - city 관계 정의 
##############################################################################################
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

In [4]:
# region_txt 의 max length 체크 

sql = "select max(char_length(region_txt)) from origin_terror_data;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(27,)]

- Region 테이블 만들기 
```
mysql> create table Region(
    ->     region_code int not null auto_increment primary key,
    ->     region_name varchar(32)
    -> );
```

In [5]:

sql = "select max(char_length(Country_txt)) from origin_terror_data;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(32,)]

- Country 테이블 만들기 
```
mysql> create table Country(
    ->     country_code int not null auto_increment primary key,
    ->     region_code int,
    ->     country_name varchar(33),
    ->     constraint region foreign key (region_code) references Region(region_code)
    -> );
```

문제 2-2. origin_terror_data 테이블에서 Region 및 Country 데이터를 추출하여 문제 2-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요. 

제출 3. Table 생성 및 조회 결과 : DESC Region, DESC Country

제출 4. 관련 코드 및 실행 결과 (ipynb)

In [12]:
sql = "select distinct region, region_txt FROM origin_terror_data order by region asc;;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(1, 'North America'),
 (2, 'Central America & Caribbean'),
 (3, 'South America'),
 (4, 'East Asia'),
 (5, 'Southeast Asia'),
 (6, 'South Asia'),
 (7, 'Central Asia'),
 (8, 'Western Europe'),
 (9, 'Eastern Europe'),
 (10, 'Middle East & North Africa'),
 (11, 'Sub-Saharan Africa'),
 (12, 'Australasia & Oceania')]

In [16]:
# 데이터 삽입을 위한 SQL 쿼리
insert_sql = "INSERT INTO Region (region_code, region_name) VALUES (%s, %s)"

# 여러 개의 데이터를 한 번에 삽입
cursor.executemany(insert_sql, result)

# 변경 사항을 커밋
conn.commit()

# 연결 닫기
cursor.close()
conn.close()

In [22]:
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)
# Region 테이블에 INSERT 
sql = "select distinct country, region, country_txt FROM origin_terror_data order by country asc;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(4, 6, 'Afghanistan'),
 (5, 9, 'Albania'),
 (6, 10, 'Algeria'),
 (7, 8, 'Andorra'),
 (8, 11, 'Angola'),
 (10, 2, 'Antigua and Barbuda'),
 (11, 3, 'Argentina'),
 (12, 7, 'Armenia'),
 (14, 12, 'Australia'),
 (15, 8, 'Austria'),
 (16, 7, 'Azerbaijan'),
 (17, 2, 'Bahamas'),
 (18, 10, 'Bahrain'),
 (19, 6, 'Bangladesh'),
 (20, 2, 'Barbados'),
 (21, 8, 'Belgium'),
 (22, 2, 'Belize'),
 (23, 11, 'Benin'),
 (25, 6, 'Bhutan'),
 (26, 3, 'Bolivia'),
 (28, 9, 'Bosnia-Herzegovina'),
 (29, 11, 'Botswana'),
 (30, 3, 'Brazil'),
 (31, 5, 'Brunei'),
 (32, 9, 'Bulgaria'),
 (33, 11, 'Burkina Faso'),
 (34, 11, 'Burundi'),
 (35, 9, 'Belarus'),
 (36, 5, 'Cambodia'),
 (37, 11, 'Cameroon'),
 (38, 1, 'Canada'),
 (41, 11, 'Central African Republic'),
 (42, 11, 'Chad'),
 (43, 3, 'Chile'),
 (44, 4, 'China'),
 (45, 3, 'Colombia'),
 (46, 11, 'Comoros'),
 (47, 11, 'Republic of the Congo'),
 (49, 2, 'Costa Rica'),
 (50, 9, 'Croatia'),
 (51, 2, 'Cuba'),
 (53, 8, 'Cyprus'),
 (54, 9, 'Czech Republic'),
 (55, 8, 'Denmark')

In [24]:
# 데이터 삽입을 위한 SQL 쿼리
insert_sql = "INSERT INTO Country (country_code, region_code, country_name) VALUES (%s, %s, %s)"

# 여러 개의 데이터를 한 번에 삽입
cursor.executemany(insert_sql, result)

# 변경 사항을 커밋
conn.commit()

# 연결 닫기
cursor.close()
conn.close()

```
mysql> select * from Country limit 10;
+--------------+-------------+---------------------+
| country_code | region_code | country_name        |
+--------------+-------------+---------------------+
|            4 |           6 | Afghanistan         |
|            5 |           9 | Albania             |
|            6 |          10 | Algeria             |
|            7 |           8 | Andorra             |
|            8 |          11 | Angola              |
|           10 |           2 | Antigua and Barbuda |
|           11 |           3 | Argentina           |
|           12 |           7 | Armenia             |
|           14 |          12 | Australia           |
|           15 |           8 | Austria             |
+--------------+-------------+--------
```-------------+       |
|         1003 |           9 | Kosovo                           |
|         1004 |          11 | South Sudan                      |
+--------------+-------------+----------------------------------+

문제 3.  origin_terror_data 에서 attack type, target type, weapon type 관련 데이터는 code 와 txt 속성으로 정의되어 있습니다.  

문제 3-1. AttackType, TargetType, WeaponType 테이블을 그림과 같은 구조로 생성하세요. 

- origin_terror_data 를 분석하여 각 테이블의 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고>
- AttackType.attacktype_code    = origin_terror_data.attacktype1
- AttackType.attacktype_desc    = origin_terror_data.attacktype1_txt 
- TargetType.targtype_code      = origin_terror_data.targtype1
- TargetType.targtype_desc      = origin_terror_data.targtype1_txt
- WeaponType.weaptype_code      = origin_terror_data.weaptype1
- WeaponType.weaptype_desc      = origin_terror_data.weaptype1_txt 

In [52]:
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

In [53]:
# attacktype1 의 max length 체크 
sql = "select max(char_length(attacktype1_txt)) from origin_terror_data;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(35,)]

```
+-----------------+-------------+------+-----+---------+----------------+
| Field           | Type        | Null | Key | Default | Extra          |
+-----------------+-------------+------+-----+---------+----------------+
| attacktype_code | int         | NO   | PRI | NULL    | auto_increment |
| attacktype_desc | varchar(36) | YES  |     | NULL    |                |
+-----------------+-------------+------+-----+---------+----------------+

In [33]:
# targettype1 의 max length 체크 
sql = "select max(char_length(targtype1_txt)) from origin_terror_data;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(30,)]

```
+---------------+-------------+------+-----+---------+----------------+
| Field         | Type        | Null | Key | Default | Extra          |
+---------------+-------------+------+-----+---------+----------------+
| targtype_code | int         | NO   | PRI | NULL    | auto_increment |
| targtype_desc | varchar(36) | YES  |     | NULL    |                |
+---------------+-------------+------+-----+---------+----------------+
```

In [34]:
# weaptype1 의 max length 체크 
sql = "select max(char_length(weaptype1_txt)) from origin_terror_data;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(75,)]

```
+---------------+-------------+------+-----+---------+----------------+
| Field         | Type        | Null | Key | Default | Extra          |
+---------------+-------------+------+-----+---------+----------------+
| weaptype_code | int         | NO   | PRI | NULL    | auto_increment |
| weaptype_desc | varchar(76) | YES  |     | NULL    |                |
+---------------+-------------+------+-----+---------+----------------+
```

문제 3-2. origin_terror_data 테이블에서 Attack Type, Target Type, Weapon Type 데이터를 추출하여 문제 3-1.에서 생성한 테이블에 입력하고 확인하세요.   

- 중복을 제거한 Unique Data 를 추출하세요. 
- 데이터를 INSERT 할때 순서를 고민하세요.

제출 5. Table 생성 및 조회 결과 : DESC AttackType, DESC TargetType, WeaponType

제출 6. 관련 코드 및 실행 결과 (ipynb) 

In [35]:
# attacktype1 데이터 추출하기 
sql = "select distinct attacktype1, attacktype1_txt FROM origin_terror_data order by attacktype1 asc;;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(1, 'Assassination'),
 (2, 'Armed Assault'),
 (3, 'Bombing/Explosion'),
 (4, 'Hijacking'),
 (5, 'Hostage Taking (Barricade Incident)'),
 (6, 'Hostage Taking (Kidnapping)'),
 (7, 'Facility/Infrastructure Attack'),
 (8, 'Unarmed Assault'),
 (9, 'Unknown')]

In [36]:
# 데이터 삽입을 위한 SQL 쿼리
insert_sql = "INSERT INTO AttackType (attacktype_code, attacktype_desc) VALUES (%s, %s)"

# 여러 개의 데이터를 한 번에 삽입
cursor.executemany(insert_sql, result)

# 변경 사항을 커밋
conn.commit()

# # 연결 닫기
# cursor.close()
# conn.close()

In [39]:
# attacktype1 데이터 추출하기 
sql = "select distinct targtype1, targtype1_txt FROM origin_terror_data order by targtype1 asc;;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(1, 'Business'),
 (2, 'Government (General)'),
 (3, 'Police'),
 (4, 'Military'),
 (5, 'Abortion Related'),
 (6, 'Airports & Aircraft'),
 (7, 'Government (Diplomatic)'),
 (8, 'Educational Institution'),
 (9, 'Food or Water Supply'),
 (10, 'Journalists & Media'),
 (11, 'Maritime'),
 (12, 'NGO'),
 (13, 'Other'),
 (14, 'Private Citizens & Property'),
 (15, 'Religious Figures/Institutions'),
 (16, 'Telecommunication'),
 (17, 'Terrorists/Non-State Militia'),
 (18, 'Tourists'),
 (19, 'Transportation'),
 (20, 'Unknown'),
 (21, 'Utilities'),
 (22, 'Violent Political Party')]

In [41]:
# 데이터 삽입을 위한 SQL 쿼리
insert_sql = "INSERT INTO TargetType (targtype_code, targtype_desc) VALUES (%s, %s)"

# 여러 개의 데이터를 한 번에 삽입
cursor.executemany(insert_sql, result)

# 변경 사항을 커밋
conn.commit()

In [42]:
# weaptype1 데이터 추출하기 
sql = "select distinct weaptype1, weaptype1_txt FROM origin_terror_data order by weaptype1 asc;;"

cursor.execute(sql)
result = cursor.fetchall()
result

[(1, 'Biological'),
 (2, 'Chemical'),
 (3, 'Radiological'),
 (5, 'Firearms'),
 (6, 'Explosives'),
 (7, 'Fake Weapons'),
 (8, 'Incendiary'),
 (9, 'Melee'),
 (10,
  'Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)'),
 (11, 'Sabotage Equipment'),
 (12, 'Other'),
 (13, 'Unknown')]

In [47]:
# 데이터 삽입을 위한 SQL 쿼리
insert_sql = "INSERT INTO WeaponType (weaptype_code, weaptype_desc) VALUES (%s, %s)"

# 여러 개의 데이터를 한 번에 삽입
cursor.executemany(insert_sql, result)

# 변경 사항을 커밋
conn.commit()

In [45]:
# 연결 닫기
cursor.close()
conn.close()

문제 4. TerrorData 테이블을 만들고 앞서 만들어둔 테이블과 관계를 설정하도록 하겠습니다. 
문제 4-1. TerrorData 테이블을 앞의 그림과 같이 생성하세요. 

- origin_terror_data 를 분석하여 데이터 타입을 정의하세요. 
- 문자열 데이터의 사이즈는 origin_terror_data 테이블에서 해당 데이터의 max length 를 쿼리로 체크하여 정의하세요. 

참고> 
- TerrorData.terror id : Auto Increment 
- TerrorData.city_name    = origin_terror_data.city
- TerrorData.target       = orgin_terror_data.target1
- TerrorData.group_name   = origin_terror_data.gname
- TerrorData.kill_count   = origin_terror_data.nkill
- TerrorData.wound_count  = origin_terror_data.nwound
- TerrorData.motive       = origin_terror_data.motive
- TerrorData.summary      = origin_terror_data.summary
- TerrorData.latitude     = origin_terror_data_latitude
- TerrorData.longitude    = origin_terror_data_longitude
- TerrorData.terror_date  = origin_terror_data.iyear + origin_terror_Data.imonth + origin_terror_data.iday (Date Type)

In [80]:
# city의 max length 체크
sql_lng = "SELECT MAX(CHAR_LENGTH(longitude)) FROM origin_terror_data;"
cursor.execute(sql_lng)
max_length_lng = cursor.fetchone()[0]
print(f"city: {max_length_lng}")

city: 11


In [81]:
# city의 max length 체크
sql_lat = "SELECT MAX(CHAR_LENGTH(latitude)) FROM origin_terror_data;"
cursor.execute(sql_lat)
max_length_lat = cursor.fetchone()[0]
print(f"city: {max_length_lat}")

city: 10


In [50]:
# 데이터베이스 연결 설정
remote = mysql.connector.connect(
    host='localhost',
    port=3306,
    user='root',
    password='mint',
    database='oneday'
)

cursor = remote.cursor(buffered=True)


# city의 max length 체크
sql_city = "SELECT MAX(CHAR_LENGTH(city)) FROM origin_terror_data;"
cursor.execute(sql_city)
max_length_city = cursor.fetchone()[0]
print(f"city: {max_length_city}")

# target1의 max length 체크
sql_target1 = "SELECT MAX(CHAR_LENGTH(target1)) FROM origin_terror_data;"
cursor.execute(sql_target1)
max_length_target1 = cursor.fetchone()[0]
print(f"target1: {max_length_target1}")

# gname의 max length 체크
sql_gname = "SELECT MAX(CHAR_LENGTH(gname)) FROM origin_terror_data;"
cursor.execute(sql_gname)
max_length_gname = cursor.fetchone()[0]
print(f"gname: {max_length_gname}")

# summary의 max length 체크
sql_summary = "SELECT MAX(CHAR_LENGTH(summary)) FROM origin_terror_data;"
cursor.execute(sql_summary)
max_length_summary = cursor.fetchone()[0]
print(f"summary: {max_length_summary}")

# motive의 max length 체크
sql_motive = "SELECT MAX(CHAR_LENGTH(motive)) FROM origin_terror_data;"
cursor.execute(sql_motive)
max_length_motive = cursor.fetchone()[0]
print(f"motive: {max_length_motive}")

# 연결 닫기
cursor.close()
remote.close()

city: 65
target1: 343
gname: 112
summary: 2431
motive: 899


```
TerrorData 테이블 만들기 
CREATE TABLE TerrorData (
    terror_id INT AUTO_INCREMENT PRIMARY KEY,
    region_code INT NOT NULL,
    country_code INT NOT NULL,
    attacktype_code INT NOT NULL,
    targtype_code INT NOT NULL,
    weaptype_code INT NOT NULL,

    terror_date DATE NOT NULL,
    city VARCHAR(66),
    target VARCHAR(350),
    group_name VARCHAR(120),
    kill_count INT NOT NULL DEFAULT 0,
    wound_count INT NOT NULL DEFAULT 0,
    motive VARCHAR(900),
    summary VARCHAR(2450),
    latitude DECIMAL(16, 14),
    longitude DECIMAL(17, 14),

    constraint region2 foreign key (region_code) references Region(region_code),
    constraint country2 foreign key (country_code) references Country(country_code),
    constraint attacktype2 foreign key (attacktype_code) references AttackType(attacktype_code),
    constraint targtype2 foreign key (targtype_code) references TargetType(targtype_code),
    constraint weaptype2 foreign key (weaptype_code) references WeaponType(weaptype_code)
);

```
+-----------------+----------------+------+-----+---------+----------------+
| Field           | Type           | Null | Key | Default | Extra          |
+-----------------+----------------+------+-----+---------+----------------+
| terror_id       | int            | NO   | PRI | NULL    | auto_increment |
| region_code     | int            | NO   | MUL | NULL    |                |
| country_code    | int            | NO   | MUL | NULL    |                |
| attacktype_code | int            | NO   | MUL | NULL    |                |
| targtype_code   | int            | NO   | MUL | NULL    |                |
| weaptype_code   | int            | NO   | MUL | NULL    |                |
| terror_date     | date           | NO   |     | NULL    |                |
| city            | varchar(66)    | YES  |     | NULL    |                |
| target          | varchar(350)   | YES  |     | NULL    |                |
| group_name      | varchar(120)   | YES  |     | NULL    |                |
| kill_count      | int            | NO   |     | 0       |                |
| wound_count     | int            | NO   |     | 0       |                |
| motive          | varchar(900)   | YES  |     | NULL    |                |
| summary         | varchar(2450)  | YES  |     | NULL    |                |
| latitude        | decimal(16,14) | YES  |     | NULL    |                |
| longitude       | decimal(17,14) | YES  |     | NULL    |                |
+-----------------+----------------+------+-----+--------
```-+----------------+ows in set (0.01 sec)

문제 4-2. origin_terror_data 테이블에서 Terror Data를 추출하여 문제 4-1.에서 생성한 테이블에 입력하고 확인하세요.

- 앞서 생성한 Region, Country, AttackType, TargetType, WeaponType 데이터와의 관계에 주의하세요.
- Count 값을 가지는 칼럼의 값이 null 인 경우, 0으로 예외처리 해주세요.
- 위도 경도 데이터 중 범위를 넘어서는 데이터가 존재합니다. 이 경우, null 값으로 예외처리 해주세요. (위도 경도 범위 : 구글링해보세요.)
- terror_date 칼럼의 경우, origin_terror_data 의 연, 월, 일 정보를 조합하여 date type 으로 정의해주세요. (Format : ‘YYYY-mm-dd’)
- origin_terror_data 의 월, 일 정보중 값이 0 인 경우 date type 으로 변환되지 않습니다. 이 경우, 1 로 예외처리 해주세요.
- 데이터 입력까지 완료한 이후, origin_terror_data 테이블을 삭제하고 확인하세요.

제출 7. Table 생성 및 조회 결과 : DESC TerrorData, SELECT COUNT(*) FROM TerrorData 

제출 8. 관련 코드 및 실행 결과 (ipynb)

In [8]:
sql = "INSERT INTO TerrorData (\
        region_code, country_code, attacktype_code, targtype_code, weaptype_code,\
        city, target, group_name, kill_count, wound_count, motive, summary,\
        latitude, longitude, terror_date)\
     select region, country, attacktype1, targtype1,  weaptype1, city,  target1,  gname, \
     IF(nkill IS NULL, 0, nkill) AS nkill,\
     IF(nwound IS NULL, 0, nwound) AS nwound,\
     motive, summary,\
    CASE WHEN latitude BETWEEN -90 AND 90 THEN NULL ELSE latitude END AS latitude,\
    CASE WHEN longitude BETWEEN -180 AND 180 THEN longitude ELSE NULL END AS longitude,\
     STR_TO_DATE(\
         CONCAT(IF(iyear = 0, 1, iyear), '-', IF(imonth = 0, 1, imonth), '-', IF(iday = 0, 1, iday) ), '%Y-%m-%d') AS event_date\
    FROM origin_terror_data;"

cursor.execute(sql)
conn.commit()

In [3]:
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

In [10]:
# 연결 닫기
cursor.close()
# remote.close()

True

In [9]:
# TerrorData 테이블에 INSERT 
sql = 'select count(*) from TerrorData'
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[(181691,)]


문제 5. TerrorData 의 전체 기간에서 테러의 숫자를 연도별로 집계하여 연도별 테러 발생 건수를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 

제출 9. 관련 코드 및 실행 결과 (ipynb)

In [12]:
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

In [16]:
num_fileds = len(cursor.description)
field_names = [i[0] for i in cursor.description]
print(field_names)

['terror_id', 'region_code', 'country_code', 'attacktype_code', 'targtype_code', 'weaptype_code', 'terror_date', 'city', 'target', 'group_name', 'kill_count', 'wound_count', 'motive', 'summary', 'latitude', 'longitude']


In [23]:
# TerrorData 테이블에 INSERT 
sql = 'SELECT YEAR(terror_date) AS terror_year, COUNT(*) AS record_count FROM TerrorData GROUP BY terror_year ORDER BY terror_year;'
cursor.execute(sql)
result = cursor.fetchall()
result

[(1970, 651),
 (1971, 471),
 (1972, 568),
 (1973, 473),
 (1974, 581),
 (1975, 740),
 (1976, 923),
 (1977, 1319),
 (1978, 1526),
 (1979, 2662),
 (1980, 2662),
 (1981, 2586),
 (1982, 2544),
 (1983, 2870),
 (1984, 3495),
 (1985, 2915),
 (1986, 2860),
 (1987, 3183),
 (1988, 3721),
 (1989, 4324),
 (1990, 3887),
 (1991, 4683),
 (1992, 5071),
 (1994, 3456),
 (1995, 3081),
 (1996, 3058),
 (1997, 3197),
 (1998, 934),
 (1999, 1395),
 (2000, 1814),
 (2001, 1906),
 (2002, 1333),
 (2003, 1278),
 (2004, 1166),
 (2005, 2017),
 (2006, 2758),
 (2007, 3242),
 (2008, 4805),
 (2009, 4721),
 (2010, 4826),
 (2011, 5076),
 (2012, 8522),
 (2013, 12036),
 (2014, 16903),
 (2015, 14965),
 (2016, 13587),
 (2017, 10900)]

문제 6. TerrorData 에서 테러가 가장 많이 일어난 순서로 국가를 정렬하여 상위 10위 국가를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 10. 관련 코드 및 실행 결과 (ipynb)

In [31]:
# 테러가 많이 일어난 상위 10위 Region
sql = 'SELECT Region.region_name, COUNT(*) AS record_count FROM\
        TerrorData JOIN Region ON TerrorData.region_code = Region.region_code GROUP BY TerrorData.region_code,\
        Region.region_name ORDER BY record_count DESC limit 10;'
cursor.execute(sql)
result = cursor.fetchall()
result

[('Middle East & North Africa', 50474),
 ('South Asia', 44974),
 ('South America', 18978),
 ('Sub-Saharan Africa', 17550),
 ('Western Europe', 16639),
 ('Southeast Asia', 12485),
 ('Central America & Caribbean', 10344),
 ('Eastern Europe', 5144),
 ('North America', 3456),
 ('East Asia', 802)]

In [3]:
conn = mysql.connector.connect(
    host = "localhost",
    port = 3306,
    user = "root",
    password = "mint",
    database = "oneday "
)
cursor = conn.cursor(buffered=True)

In [32]:
# 테러가 많이 일어난 상위 10위 Country 
sql = 'SELECT Country.country_name, COUNT(*) AS record_count FROM\
        TerrorData JOIN Country ON TerrorData.country_code = Country.country_code GROUP BY TerrorData.country_code,\
        Country.country_name ORDER BY record_count DESC limit 10;'
cursor.execute(sql)
result = cursor.fetchall()
result

[('Iraq', 24636),
 ('Pakistan', 14368),
 ('Afghanistan', 12731),
 ('India', 11960),
 ('Colombia', 8306),
 ('Philippines', 6908),
 ('Peru', 6096),
 ('El Salvador', 5320),
 ('United Kingdom', 5235),
 ('Turkey', 4292)]

문제 7. TerrorData 에서 테러가 가장 많이 일어난 상위 10위 국가에 대해 국가별로 사망자수와 부상자수, 사상자수(사망자수 + 부상자수)를 조회하세요. 

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 국가 정보는 이름으로 출력하세요.

제출 11. 관련 코드 및 실행 결과 (ipynb)

In [7]:
# 테러가 많이 일어난 상위 10위 Country 
sql = 'SELECT Country.country_name AS country, SUM(kill_count) AS total_kills, SUM(wound_count) AS total_wounds, SUM(kill_count + wound_count) \
        AS total_casualties FROM TerrorData JOIN Country ON TerrorData.country_code = Country.country_code \
        GROUP BY TerrorData.country_code, Country.country_name ORDER BY total_casualties DESC LIMIT 10;'
cursor.execute(sql)
result = cursor.fetchall()
result

[('Iraq', Decimal('78589'), Decimal('134690'), Decimal('213279')),
 ('Afghanistan', Decimal('39384'), Decimal('44277'), Decimal('83661')),
 ('Pakistan', Decimal('23822'), Decimal('42038'), Decimal('65860')),
 ('India', Decimal('19341'), Decimal('28980'), Decimal('48321')),
 ('Nigeria', Decimal('22682'), Decimal('10238'), Decimal('32920')),
 ('Sri Lanka', Decimal('15530'), Decimal('15561'), Decimal('31091')),
 ('Syria', Decimal('15229'), Decimal('14109'), Decimal('29338')),
 ('Colombia', Decimal('14698'), Decimal('10328'), Decimal('25026')),
 ('United States', Decimal('3771'), Decimal('20702'), Decimal('24473')),
 ('Philippines', Decimal('9559'), Decimal('13367'), Decimal('22926'))]

문제 8. 지역별 테러 공격 형태에 따른 사망자수, 부상자수, 사상자수를 조회하세요.

- Python 코드에서 SQL 쿼리를 사용하여 조회합니다. 
- 지역 정보(Region)는 이름으로 출력하세요.

제출 12. 관련 코드 및 실행 결과 (ipynb)

In [13]:
# 테러가 많이 일어난 상위 10위 Country 
sql = 'SELECT Region.region_name AS region, AttackType.attacktype_desc AS attack_desc, SUM(TerrorData.kill_count) AS total_kills, \
    SUM(TerrorData.wound_count) AS total_wounds, SUM(TerrorData.kill_count + TerrorData.wound_count) AS total_casualties\
    FROM TerrorData JOIN Region ON TerrorData.region_code = Region.region_code JOIN AttackType ON TerrorData.attacktype_code = AttackType.attacktype_code\
    GROUP BY Region.region_name, AttackType.attacktype_desc;'
cursor.execute(sql)
result = cursor.fetchall()
result

[('Central America & Caribbean',
  'Assassination',
  Decimal('1699'),
  Decimal('246'),
  Decimal('1945')),
 ('Southeast Asia',
  'Assassination',
  Decimal('1632'),
  Decimal('821'),
  Decimal('2453')),
 ('South America',
  'Assassination',
  Decimal('4239'),
  Decimal('655'),
  Decimal('4894')),
 ('North America',
  'Assassination',
  Decimal('238'),
  Decimal('84'),
  Decimal('322')),
 ('Middle East & North Africa',
  'Assassination',
  Decimal('4774'),
  Decimal('4521'),
  Decimal('9295')),
 ('South Asia',
  'Assassination',
  Decimal('6701'),
  Decimal('5180'),
  Decimal('11881')),
 ('Western Europe',
  'Assassination',
  Decimal('2673'),
  Decimal('801'),
  Decimal('3474')),
 ('Sub-Saharan Africa',
  'Assassination',
  Decimal('2371'),
  Decimal('1082'),
  Decimal('3453')),
 ('Australasia & Oceania',
  'Assassination',
  Decimal('34'),
  Decimal('9'),
  Decimal('43')),
 ('East Asia', 'Assassination', Decimal('44'), Decimal('35'), Decimal('79')),
 ('Eastern Europe',
  'Assassinat

---

# reference

민형동 (2020). Global Terrorism Data를 활용한 국가별 테러유형 분석. 한국공안행정학회, 82, 63-84. 

한인택 (2017). Global Terrorism Database를 활용한 테러리즘의 현황분석. 

민형기 (2019). 전세계 테러 데이터 분석 (따라하기). https://pinkwink.kr/1214 

ASHWINI SWSAIN (2018). Terrorism Around The World. https://www.kaggle.com/code/ash316/terrorism-around-the-world/notebook

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021). *Global Terrorism Database* (GTD) [Data set]. University of Maryland. https://www.start.umd.edu/gtd

START (National Consortium for the Study of Terrorism and Responses to Terrorism). (2021, August). *Global Terrorism Database codebook: Methodology, inclusion criteria, and variables.* University of Maryland. https://www.start.umd.edu/gtd/downloads/Codebook.pdf 
